## kaggle鐵達尼號專題（專題1）
    
### 評分標準：Kaggle分數再開根號乘以十（四捨五入）
e.x. Kaggle正確率76 >> 最終分數87

注意事項
<li>需上傳ipynb檔以及kaggle的分數截圖</li>
<li>若正確率100%將不計分</li>
<li>不得使用別人寫好的程式碼</li>

#### 第0步：下載資料 (https://www.kaggle.com/c/titanic)

In [1]:
import pandas as pd
df = pd.read_csv('train.csv')
train_df = df.drop(['Survived'],axis=1)
#target = df.Survived.to_numpy()
train_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


#### 第1步：資料處理、模型製作、訓練

In [2]:
train_df = df.drop(['Name'],axis=1)
train_df = train_df.drop(['Cabin'],axis=1)
train_df = train_df.drop(['Ticket'],axis=1)
train_df = train_df.drop(['PassengerId'],axis=1)
target = train_df.Survived.to_numpy()
train_df = train_df.drop(['Survived'],axis=1)
train_df = train_df.fillna(0)
train_df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S


In [3]:
train_df['Sex'] = train_df['Sex'].astype('category').cat.codes
train_df['Embarked'] = train_df['Embarked'].astype('category').cat.codes
train_df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,22.0,1,0,7.2500,3
1,1,0,38.0,1,0,71.2833,1
2,3,0,26.0,0,0,7.9250,3
3,1,0,35.0,1,0,53.1000,3
4,3,1,35.0,0,0,8.0500,3


In [4]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
for x in train_df.columns:
    train_df[x]=scaler.fit_transform(train_df[x].to_frame())
train_df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1.0,1.0,0.2750,0.125,0.0,0.014151,1.000000
1,0.0,0.0,0.4750,0.125,0.0,0.139136,0.333333
2,1.0,0.0,0.3250,0.000,0.0,0.015469,1.000000
3,0.0,0.0,0.4375,0.125,0.0,0.103644,1.000000
4,1.0,1.0,0.4375,0.000,0.0,0.015713,1.000000


In [5]:
train_df.to_csv('test1.csv',index=False)

In [6]:
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_df, target, test_size=0.25,random_state=1)

In [7]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier
model_dt = DecisionTreeClassifier(random_state=1,criterion='entropy')
model_dt.fit(X_train, y_train)
model_dt.score(X_test, y_test)

0.7757847533632287

In [8]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier(random_state=1,criterion='entropy',max_depth=2)
model_rf.fit(X_train, y_train)
model_rf.score(X_test, y_test)

0.757847533632287

In [9]:
#Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier
model_gb = GradientBoostingClassifier(random_state=1)
model_gb.fit(X_train, y_train)
model_gb.score(X_test, y_test)

0.7847533632286996

In [10]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression
model_lr = LogisticRegression(random_state=1)
model_lr.fit(X_train, y_train)
model_lr.score(X_test, y_test)

0.7892376681614349

In [11]:
#Support Vector Machine
from sklearn.svm import SVC
model_svc = SVC(random_state=1)
model_svc.fit(X_train, y_train)
model_svc.score(X_test, y_test)

0.7802690582959642

In [12]:
#Extreme Gradient Boosting
import xgboost as xgb
from sklearn import metrics

def auc(m, train, test): 
    return (metrics.roc_auc_score(y_train,m.predict_proba(train)[:,1]),
                            metrics.roc_auc_score(y_test,m.predict_proba(test)[:,1]))
model_xgb = xgb.XGBClassifier(max_depth=50, min_child_weight=1, n_estimators=200,verbosity=1,learning_rate=0.16)

ModuleNotFoundError: No module named 'xgboost'

In [ ]:
eval_set = [(X_test, y_test)]
model_xgb.fit(X_train, y_train, eval_metric="auc", eval_set=eval_set, verbose=True,early_stopping_rounds=2)

In [ ]:
auc(model_xgb, X_train, X_test)

In [205]:
#Light Gradient Boosting Machine
import lightgbm as lgb

model_lgb = lgb.LGBMClassifier(silent=False,max_depth=50,n_estimators=300,num_leaves=900,learning_rate=0.1)

In [206]:
model_lgb.fit(X_train,y_train, eval_metric='auc',eval_set=eval_set,
          categorical_feature = 'auto',early_stopping_rounds=2)

[LightGBM] [Info] Number of positive: 247, number of negative: 421
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000984 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 197
[LightGBM] [Info] Number of data points in the train set: 668, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.369760 -> initscore=-0.533244
[LightGBM] [Info] Start training from score -0.533244
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	valid_0's auc: 0.840173	valid_0's binary_logloss: 0.645977
Training until validation scores don't improve for 2 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]	valid_0's auc: 0.846505	valid_0's binary_logloss: 0.613074
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3]	valid_0's auc: 0.847574	valid_0'

LGBMClassifier(max_depth=50, n_estimators=300, num_leaves=900, silent=False)

In [207]:
auc(model_lgb, X_train, X_test)

(0.9072480213872888, 0.8475740131578948)

In [15]:
#Categorical Boosting
import catboost as cb
cat_features_index = ["Pclass","Sex","Embarked"]

model_cb = cb.CatBoostClassifier(eval_metric="AUC",one_hot_max_size=31, \
                            depth=10, iterations= 500, l2_leaf_reg= 9, learning_rate= 0.15)

In [17]:
eval_set = [(X_test, y_test)]
model_cb.fit(X_train,y_train,eval_set=eval_set,early_stopping_rounds=2)

0:	test: 0.8144326	best: 0.8144326 (0)	total: 55.3ms	remaining: 27.6s
1:	test: 0.8321957	best: 0.8321957 (1)	total: 56.4ms	remaining: 14s
2:	test: 0.8332237	best: 0.8332237 (2)	total: 57.8ms	remaining: 9.57s
3:	test: 0.8327303	best: 0.8332237 (2)	total: 58.3ms	remaining: 7.23s
4:	test: 0.8306332	best: 0.8332237 (2)	total: 59.9ms	remaining: 5.93s
Stopped by overfitting detector  (2 iterations wait)

bestTest = 0.8332236842
bestIteration = 2

Shrink model to first 3 iterations.


In [59]:
train_cb = train_df
for x in ['Age','SibSp','Parch','Fare']:
    train_cb[x]=scaler.fit_transform(train_cb[x].to_frame())
# for x in ['Pclass','Sex','Embarked']:
#     train_cb[x] = train_cb[x].astype(int)
train_cb.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1.0,1.0,0.2750,0.125,0.0,0.014151,1.0
1,0.0,0.0,0.4750,0.125,0.0,0.139136,0.3333333333333333
2,1.0,0.0,0.3250,0.000,0.0,0.015469,1.0
3,0.0,0.0,0.4375,0.125,0.0,0.103644,1.0
4,1.0,1.0,0.4375,0.000,0.0,0.015713,1.0


In [60]:
Pclassl = []
Sexl = []
Embarkedl = []
for x in train_cb['Pclass']:
    Pclassl.append(str(int(float(x))))
for x in train_cb['Sex']:
    Sexl.append(str(int(float(x))))
for x in train_cb['Embarked']:
    Embarkedl.append(str(int(float(x))))
train_cb['Pclass']=Pclassl
train_cb['Sex']=Sexl
train_cb['Embarked']=Embarkedl
train_cb

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,1,0.2750,0.125,0.000000,0.014151,1
1,0,0,0.4750,0.125,0.000000,0.139136,0
2,1,0,0.3250,0.000,0.000000,0.015469,1
3,0,0,0.4375,0.125,0.000000,0.103644,1
4,1,1,0.4375,0.000,0.000000,0.015713,1
...,...,...,...,...,...,...,...
886,0,1,0.3375,0.000,0.000000,0.025374,1
887,0,0,0.2375,0.000,0.000000,0.058556,1
888,1,0,0.0000,0.125,0.333333,0.045771,1
889,0,1,0.3250,0.000,0.000000,0.058556,0


In [61]:
X_traincb, X_testcb, y_traincb, y_testcb = train_test_split(train_cb, target, test_size=0.25,random_state=1)

In [63]:
X_traincb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 668 entries, 35 to 37
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    668 non-null    object 
 1   Sex       668 non-null    object 
 2   Age       668 non-null    float64
 3   SibSp     668 non-null    float64
 4   Parch     668 non-null    float64
 5   Fare      668 non-null    float64
 6   Embarked  668 non-null    object 
dtypes: float64(4), object(3)
memory usage: 41.8+ KB


In [71]:
model_cb.fit(X_traincb,y_traincb, cat_features=[0,1,6],early_stopping_rounds=2)

0:	total: 916us	remaining: 458ms
1:	total: 1.89ms	remaining: 470ms
2:	total: 3.11ms	remaining: 516ms
3:	total: 3.82ms	remaining: 474ms
4:	total: 8.78ms	remaining: 869ms
5:	total: 9.06ms	remaining: 746ms
6:	total: 9.42ms	remaining: 663ms
7:	total: 14.9ms	remaining: 916ms
8:	total: 18.8ms	remaining: 1.02s
9:	total: 20.1ms	remaining: 983ms
10:	total: 24.3ms	remaining: 1.08s
11:	total: 25ms	remaining: 1.02s
12:	total: 26.3ms	remaining: 985ms
13:	total: 27.5ms	remaining: 953ms
14:	total: 31.8ms	remaining: 1.03s
15:	total: 35.5ms	remaining: 1.07s
16:	total: 39.6ms	remaining: 1.13s
17:	total: 44.1ms	remaining: 1.18s
18:	total: 47.9ms	remaining: 1.21s
19:	total: 51.5ms	remaining: 1.24s
20:	total: 53.8ms	remaining: 1.23s
21:	total: 58.7ms	remaining: 1.27s
22:	total: 62.5ms	remaining: 1.3s
23:	total: 66.1ms	remaining: 1.31s
24:	total: 67.6ms	remaining: 1.28s
25:	total: 70.8ms	remaining: 1.29s
26:	total: 75.5ms	remaining: 1.32s
27:	total: 79.1ms	remaining: 1.33s
28:	total: 79.9ms	remaining: 1.3s


#### 第2步：載入測試集、資料處理、預測

In [215]:
test_df = pd.read_csv('titanic/test.csv')
##資料處理

# pred = model.predict(test_df)

In [216]:
test_df = test_df.drop(['Name'],axis=1)
test_df = test_df.drop(['Cabin'],axis=1)
test_df = test_df.drop(['Ticket'],axis=1)
test_df = test_df.drop(['PassengerId'],axis=1)
test_df = test_df.fillna(0)
test_df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,34.5,0,0,7.8292,Q
1,3,female,47.0,1,0,7.0000,S
2,2,male,62.0,0,0,9.6875,Q
3,3,male,27.0,0,0,8.6625,S
4,3,female,22.0,1,1,12.2875,S


In [217]:
test_df['Sex'] = test_df['Sex'].astype('category').cat.codes
test_df['Embarked'] = test_df['Embarked'].astype('category').cat.codes
test_df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,34.5,0,0,7.8292,1
1,3,0,47.0,1,0,7.0000,2
2,2,1,62.0,0,0,9.6875,1
3,3,1,27.0,0,0,8.6625,2
4,3,0,22.0,1,1,12.2875,2


In [218]:
for x in test_df.columns:
    test_df[x]=scaler.fit_transform(test_df[x].to_frame())
test_df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1.0,1.0,0.453947,0.000,0.000000,0.015282,0.5
1,1.0,0.0,0.618421,0.125,0.000000,0.013663,1.0
2,0.5,1.0,0.815789,0.000,0.000000,0.018909,0.5
3,1.0,1.0,0.355263,0.000,0.000000,0.016908,1.0
4,1.0,0.0,0.289474,0.125,0.111111,0.023984,1.0


In [219]:
pred_dt = model_dt.predict(test_df)
pred_rf = model_rf.predict(test_df)
pred_gb = model_gb.predict(test_df)
pred_lr = model_lr.predict(test_df)
pred_svc = model_svc.predict(test_df)
pred_xgb = model_xgb.predict(test_df)
pred_lgb = model_lgb.predict(test_df)
pred_cb = model_cb.predict(test_df)

#### 最後一步：上傳檔案

In [222]:
import numpy as np
submission_dt = pd.read_csv('titanic/gender_submission.csv')
submission_dt.Survived=pred_dt
submission_dt.to_csv('titanic/submission_dt.csv',index=False)

In [223]:
submission_rf = pd.read_csv('titanic/gender_submission.csv')
submission_rf.Survived=pred_rf
submission_rf.to_csv('titanic/submission_rf.csv',index=False)

In [224]:
submission_gb = pd.read_csv('titanic/gender_submission.csv')
submission_gb.Survived=pred_gb
submission_gb.to_csv('titanic/submission_gb.csv',index=False)

In [225]:
submission_lr = pd.read_csv('titanic/gender_submission.csv')
submission_lr.Survived=pred_lr
submission_lr.to_csv('titanic/submission_lr.csv',index=False)

In [227]:
submission_svc = pd.read_csv('titanic/gender_submission.csv')
submission_svc.Survived=pred_svc
submission_svc.to_csv('titanic/submission_svc.csv',index=False)

In [228]:
submission_xgb = pd.read_csv('titanic/gender_submission.csv')
submission_xgb.Survived=pred_xgb
submission_xgb.to_csv('titanic/submission_xgb.csv',index=False)

In [229]:
submission_lgb = pd.read_csv('titanic/gender_submission.csv')
submission_lgb.Survived=pred_lgb
submission_lgb.to_csv('titanic/submission_lgb.csv',index=False)

In [230]:
submission_cb = pd.read_csv('titanic/gender_submission.csv')
submission_cb.Survived=pred_cb
submission_cb.to_csv('titanic/submission_cb.csv',index=False)